<a href="https://colab.research.google.com/github/mohamedabdelaty74/Offers-AI/blob/main/Offers_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

class SmartProductAnalyzer:
    """محلل ذكي لأنماط الشراء والعلاقات بين المنتجات"""

    def __init__(self):
        self.product_vectors = None
        self.product_codes = None
        self.similarity_matrix = None
        self.product_info = None

    def analyze_purchase_patterns(self, df):
        """تحليل أنماط الشراء باستخدام البيانات الفعلية"""
        # إنشاء مصفوفة الإيصالات-المنتجات
        receipt_product = pd.crosstab(df['رقم الإيصال'], df['كود المنتح'])

        # تخفيض الأبعاد للكشف عن الأنماط الخفية
        n_components = min(50, receipt_product.shape[1]-1)
        svd = TruncatedSVD(n_components=n_components)
        self.product_vectors = svd.fit_transform(receipt_product.T)
        self.product_codes = receipt_product.columns

        # حساب مصفوفة التشابه
        self.similarity_matrix = cosine_similarity(self.product_vectors)

        return receipt_product

def analyze_products_ai(input_file):
    """تحليل شامل لأنماط الشراء والعلاقات بين المنتجات - نسخة محسنة"""

    try:
        print("جاري قراءة البيانات...")
        df = pd.read_excel(input_file)

        # تنظيف وإعداد البيانات
        df['كود المنتح'] = df['كود المنتح'].astype(str)

        # حساب إحصائيات المنتجات مرة واحدة في البداية
        print("حساب إحصائيات المنتجات...")
        product_stats = df.groupby('كود المنتح').agg({
            'رقم الإيصال': lambda x: len(set(x)),  # عدد الإيصالات الفريدة لكل منتج
            'بنود الطلب /المنتج.2.2': 'first',
            'نوع العرض': 'first'
        }).reset_index()

        # تخزين الإحصائيات في قاموس للوصول السريع
        product_counts = dict(zip(product_stats['كود المنتح'], product_stats['رقم الإيصال']))

        # إنشاء محلل المنتجات
        analyzer = SmartProductAnalyzer()

        # إنشاء قاموس معلومات المنتجات
        analyzer.product_info = dict(zip(
            df['كود المنتح'],
            zip(df['بنود الطلب /المنتج.2.2'], df['نوع العرض'])
        ))

        print("تحليل أنماط الشراء...")
        receipt_product = analyzer.analyze_purchase_patterns(df)

        # حساب الإحصائيات الأساسية
        total_receipts = len(df['رقم الإيصال'].unique())
        print(f"عدد الإيصالات الكلي: {total_receipts}")

        print("تحليل المنتجات المشتركة...")
        # تجميع المنتجات حسب الإيصال بطريقة أكثر كفاءة
        receipt_groups = df.groupby('رقم الإيصال')['كود المنتح'].agg(lambda x: list(set(x)))
        combinations = []

        # حساب التركيبات المشتركة
        for products in receipt_groups:
            for i in range(len(products)):
                for j in range(i+1, len(products)):
                    prod1, prod2 = sorted([products[i], products[j]])
                    combinations.append((prod1, prod2))

        combination_counts = Counter(combinations)

        print("إعداد النتائج التفصيلية...")
        results = []
        for (prod1, prod2), count in combination_counts.most_common():
            try:
                name1, offer_type1 = analyzer.product_info.get(prod1, ('غير معروف', 'غير معروف'))
                name2, offer_type2 = analyzer.product_info.get(prod2, ('غير معروف', 'غير معروف'))

                # استخدام القيم المحسوبة مسبقاً
                prod1_count = product_counts[prod1]
                prod2_count = product_counts[prod2]

                # حساب المقاييس
                support = count / total_receipts
                prod1_support = prod1_count / total_receipts
                prod2_support = prod2_count / total_receipts

                # حساب قوة العلاقة (Lift)
                lift = support / (prod1_support * prod2_support)

                # حساب درجة الثقة في العلاقة
                confidence_1_to_2 = count / prod1_count
                confidence_2_to_1 = count / prod2_count

                results.append({
                    'كود المنتج الأول': prod1,
                    'اسم المنتج الأول': name1,
                    'نوع العرض الأول': offer_type1,
                    'عدد مرات شراء المنتج الأول': prod1_count,
                    'نسبة ظهور المنتج الأول': f"{prod1_support:.2%}",

                    'كود المنتج الثاني': prod2,
                    'اسم المنتج الثاني': name2,
                    'نوع العرض الثاني': offer_type2,
                    'عدد مرات شراء المنتج الثاني': prod2_count,
                    'نسبة ظهور المنتج الثاني': f"{prod2_support:.2%}",

                    'عدد مرات الشراء معاً': count,
                    'نسبة الشراء معاً': f"{support:.2%}",

                    'قوة العلاقة (Lift)': f"{lift:.2f}",
                    'احتمال شراء الثاني مع الأول': f"{confidence_1_to_2:.2%}",
                    'احتمال شراء الأول مع الثاني': f"{confidence_2_to_1:.2%}"
                })
            except Exception as e:
                print(f"خطأ في معالجة المنتجين {prod1} و {prod2}: {str(e)}")
                continue

        print("إنشاء التقرير النهائي...")
        # إنشاء DataFrame للنتائج
        results_df = pd.DataFrame(results)

        # إنشاء اسم الملف مع التاريخ والوقت
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_file = f'market_basket_analysis_{timestamp}.xlsx'

        # حفظ النتائج في ملف Excel
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # صفحة التحليل الرئيسي
            results_df.to_excel(writer, sheet_name='تحليل المنتجات المشتركة', index=False)

            # صفحة إحصائيات المنتجات الفردية
            product_stats.columns = ['كود المنتج', 'عدد مرات الشراء', 'اسم المنتج', 'نوع العرض']
            product_stats['نسبة الظهور في الإيصالات'] = product_stats['عدد مرات الشراء'] / total_receipts
            product_stats = product_stats.sort_values('عدد مرات الشراء', ascending=False)
            product_stats.to_excel(writer, sheet_name='إحصائيات المنتجات', index=False)

            # صفحة الملخص
            summary_data = {
                'إجمالي عدد الإيصالات': [total_receipts],
                'إجمالي عدد المنتجات الفريدة': [len(df['كود المنتح'].unique())],
                'متوسط عدد المنتجات في الإيصال': [df.groupby('رقم الإيصال')['كود المنتح'].count().mean()],
                'أكبر عدد منتجات في إيصال': [df.groupby('رقم الإيصال')['كود المنتح'].count().max()],
                'تاريخ التحليل': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')]
            }
            pd.DataFrame(summary_data).to_excel(writer, sheet_name='ملخص التحليل', index=False)

        print(f"تم حفظ التحليل في الملف: {output_file}")
        return output_file

    except Exception as e:
        print(f"حدث خطأ أثناء التحليل: {str(e)}")
        return None

def main():
    """الدالة الرئيسية لتشغيل التحليل"""
    print("نظام التحليل الذكي لسلة المشتريات")
    print("=" * 40)

    try:
        input_file = input("أدخل مسار ملف الإكسل: ")
        print("\nجاري بدء التحليل...")

        output_file = analyze_products_ai(input_file)

        if output_file:
            print("\nتم إكمال التحليل بنجاح!")
            print(f"يمكنك العثور على النتائج في الملف: {output_file}")
            print("\nيحتوي الملف على ثلاث صفحات:")
            print("1. تحليل المنتجات المشتركة: تحليل تفصيلي للعلاقات بين المنتجات")
            print("2. إحصائيات المنتجات: تحليل لكل منتج على حدة")
            print("3. ملخص التحليل: إحصائيات عامة عن البيانات")
        else:
            print("\nلم يتم إكمال التحليل بسبب حدوث خطأ")

    except Exception as e:
        print(f"\nحدث خطأ غير متوقع: {str(e)}")

    print("\nشكراً لاستخدام نظام التحليل الذكي!")

if __name__ == "__main__":
    main()